In [5]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [6]:
#Load the models saved previously as pickle files
### Load the trained model, scaler pickle,onehot
model=load_model('model.h5')

## load the encoder and scaler
with open('onehot_encoder_geo.pkl','rb') as file:
    geo_encoder=pickle.load(file)

with open('label_encoder_gender.pkl', 'rb') as file:
    gender_encoder = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [14]:
# Example input data
input_data = {
    'CreditScore': [600],
    'Geography': ['France'],
    'Gender': ['Male'],
    'Age': [40],
    'Tenure':[3],
    'Balance': [60000],
    'NumOfProducts': [2],
    'HasCrCard': [1],
    'IsActiveMember': [1],
    'EstimatedSalary': [50000]
}
if input_data['Gender'][0]=='Male':
    input_data['Gender']=1;
else:
    input_data['Gender']=0;
input_df=pd.DataFrame(input_data)
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [17]:
"""All the encoders and scalers are loaded from the pickle file because Using pickle files to save and load pre-trained encoders (or models) provides several key advantages, 
especially when you're working in production environments or dealing with larger, complex machine learning workflows."""
encoded_data=geo_encoder.transform(input_df[['Geography']])
print(encoded_data)
encoded_df=pd.DataFrame(encoded_data,columns=geo_encoder.get_feature_names_out(['Geography']))
encoded_df


[[1. 0. 0.]]


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [18]:
input_df=pd.concat([input_df,encoded_df],axis=1)
input_df.drop('Geography',axis=1,inplace=True)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [19]:
## Scaling the input data
input_scaled=scaler.transform(input_df)
input_scaled

array([[-0.54204762,  0.91055421,  0.10473081, -0.68894811, -0.26230046,
         0.81966266,  0.64598061,  0.97071435, -0.88153859,  0.99828718,
        -0.57559072, -0.57779016]])

In [20]:
## Predict churn
prediction=model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step


array([[0.17141671]], dtype=float32)

In [21]:
prediction_proba = prediction[0][0]
prediction_proba

0.17141671

In [22]:
if prediction_proba > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')

The customer is not likely to churn.
